In [25]:
import pandas as pd
import re
import yaml

In [26]:
# Start by loading relevant ABC metadata files into dataframes
dpath = '../data/WMB_taxonomy_20230830/'
cluster_annotation_term = pd.read_csv(dpath + 'cluster_annotation_term.csv', sep=',')
cluster_annotation_term_set = pd.read_csv(dpath + 'cluster_annotation_term_set.csv', sep=',')
#cluster = pd.read_csv(dpath + 'cluster.csv', sep=',')
cluster_to_cluster_annotation_membership = pd.read_csv(dpath + 'cluster_to_cluster_annotation_membership.csv', sep=',')

In [91]:
cluster_annotation_term[100:110]

,label,name,cluster_annotation_term_set_label,parent_term_label,parent_term_set_label,term_set_order,term_order,cluster_annotation_term_set_name,color_hex_triplet
100,CS20230722_SUBC_057,057 NDB-SI-MA-STRv Lhx8 Gaba,CCN20230722_SUBC,CS20230722_CLAS_08,CCN20230722_CLAS,2,56,subclass,#199900
101,CS20230722_SUBC_058,058 PAL-STR Gaba-Chol,CCN20230722_SUBC,CS20230722_CLAS_08,CCN20230722_CLAS,2,57,subclass,#4DFFFA
102,CS20230722_SUBC_059,059 GPe-SI Sox6 Cyp26b1 Gaba,CCN20230722_SUBC,CS20230722_CLAS_09,CCN20230722_CLAS,2,58,subclass,#5CCC85
103,CS20230722_SUBC_060,060 OT D3 Folh1 Gaba,CCN20230722_SUBC,CS20230722_CLAS_09,CCN20230722_CLAS,2,59,subclass,#66580F
104,CS20230722_SUBC_061,061 STR D1 Gaba,CCN20230722_SUBC,CS20230722_CLAS_09,CCN20230722_CLAS,2,60,subclass,#005B66
105,CS20230722_SUBC_062,062 STR D2 Gaba,CCN20230722_SUBC,CS20230722_CLAS_09,CCN20230722_CLAS,2,61,subclass,#FF7800
106,CS20230722_SUBC_063,063 STR D1 Sema5a Gaba,CCN20230722_SUBC,CS20230722_CLAS_09,CCN20230722_CLAS,2,62,subclass,#663A2D
107,CS20230722_SUBC_064,064 STR-PAL Chst9 Gaba,CCN20230722_SUBC,CS20230722_CLAS_09,CCN20230722_CLAS,2,63,subclass,#967B09
108,CS20230722_SUBC_065,065 IA Mgp Gaba,CCN20230722_SUBC,CS20230722_CLAS_09,CCN20230722_CLAS,2,64,subclass,#00995F
109,CS20230722_SUBC_066,066 NDB-SI-ant Prdm12 Gaba,CCN20230722_SUBC,CS20230722_CLAS_11,CCN20230722_CLAS,2,65,subclass,#FF00D6


In [4]:
cluster_annotation_term_set[0:2]

,label,name,description,order
0,CCN20230722_NEUR,neurotransmitter,Clusters are assigned based on the average exp...,0
1,CCN20230722_CLAS,class,The top level of cell type definition in the m...,1


In [5]:
# Turn these into dicts for fast lookup
# turning nan to empty string better for lookups, although could cause trouble with numbers.

def df2dict(df: pd.DataFrame, key_column: str):
    return df.set_index(key_column).fillna('').to_dict(orient='index')

cluster_annotation_term_dict = df2dict(cluster_annotation_term, 'label')
cluster_annotation_term_set_dict = df2dict(cluster_annotation_term_set, 'label')

In [6]:
# Old taxonomy - use as source of accesssions for now.  These accessions will match 
old_taxonomy = pd.read_csv('../data/BDSO_taxonomy_templates/nomenclature_table_CS202212150_annotated.csv', sep=',')
old_taxonomy [0:2]

,cell_set_accession,original_label,cell_set_label,cell_set_preferred_alias,cell_set_aligned_alias,cell_set_additional_aliases,cell_set_structure,cell_set_ontology_tag,cell_set_alias_assignee,cell_set_alias_citation,taxonomy_id,child_cell_set_accessions,NT,MBA,projection,layer,CL
0,CS202212150_1,NaN,WMB 00001,0001 Car3 Glut_1,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061535,NaN,NaN,NaN,NaN
1,CS202212150_10,NaN,WMB 00010,0010 IT EP-CLA Glut_1,NaN,NaN,Brain,UBERON:0000955,Zizhen Yao,NaN,CCN202212150,NaN,GO:0061535,http://purl.obolibrary.org/obo/MBA_942 | http:...,PATO:0070034,NaN,NaN


In [7]:
import requests
clget = requests.get("http://purl.obolibrary.org/obo/cl/cl.json")
clget

<Response [200]>

In [8]:
import json
cl = json.loads(clget.content)['graphs'][0]
cl_lookup = { n['id'].replace("http://purl.obolibrary.org/obo/CL_", "CL:") : n['lbl'] 
          for n in cl['nodes'] 
          if 'lbl' in n.keys() }


In [9]:
old_taxonomy_dict = df2dict(old_taxonomy, 'cell_set_preferred_alias')

In [10]:
# old_taxonomy has CL mappings we can use
old_taxonomy[['cell_set_accession', 'cell_set_preferred_alias', 'CL',]].dropna()

,cell_set_accession,cell_set_preferred_alias,CL
4414,CS202212150_4972,4972 CB Granule Glut_1,CL:0000120
4415,CS202212150_4973,4973 CB Granule Glut_1,CL:0000120
4416,CS202212150_4974,4974 CB Granule Glut_1,CL:0000120
4417,CS202212150_4975,4975 CB Granule Glut_2,CL:0000120
4418,CS202212150_4976,4976 CB Granule Glut_2,CL:0000120
...,...,...,...
5972,CS202212150_6374,Pallium glutamatergic,CL:0000679
5973,CS202212150_6375,Subpallium GABAergic,CL:0000617
5974,CS202212150_6376,PAL-sAMY-TH-HY-MB-HB neuronal,CL:0000540
5975,CS202212150_6377,CBX-MOB-other neuronal,CL:0000540


In [11]:
# A second source of CL mappings:
with open('../data/BDSO_taxonomy_templates/taxonomy_details.yaml', 'r') as taxd_file:
    taxd=yaml.safe_load(taxd_file.read())

In [12]:
root_node_CL_mappings = { n['Node']: n['Cell_type'] for n in taxd[0]['Root_nodes']}

In [13]:
taxd[0]['Root_nodes'][0:2]

[{'Node': 'CS202212150_6347',
  'Cell_type': 'CL:0000679',
  'Location_relation': 'has_soma_location'},
 {'Node': 'CS202212150_6348',
  'Cell_type': 'CL:0000679',
  'Location_relation': 'has_soma_location'}]

In [14]:
# Mappings in both
set(root_node_CL_mappings.values()).intersection(set(old_taxonomy['CL'].dropna()))

{'CL:0000125', 'CL:0000617', 'CL:0000679'}

In [15]:
#mappings only in root node map
set(root_node_CL_mappings.values()).difference(set(old_taxonomy['CL'].dropna()))

{'CL:0000700',
 'CL:0000738',
 'CL:0000850',
 'CL:4023063',
 'CL:4023064',
 'CL:4023072'}

In [16]:
# Mappings only in taxonomy
len(set(old_taxonomy['CL'].dropna()).difference(set(root_node_CL_mappings.values())))

17

In [105]:
# Combining CL terms into a single lookup with labels

CL_mapping = df2dict(old_taxonomy[['cell_set_accession', 'CL']].dropna(), 'cell_set_accession')
CL_mapping

{'CS202212150_4972': {'CL': 'CL:0000120'},
 'CS202212150_4973': {'CL': 'CL:0000120'},
 'CS202212150_4974': {'CL': 'CL:0000120'},
 'CS202212150_4975': {'CL': 'CL:0000120'},
 'CS202212150_4976': {'CL': 'CL:0000120'},
 'CS202212150_5099': {'CL': 'CL:0000121'},
 'CS202212150_5102': {'CL': 'CL:0000644'},
 'CS202212150_5103': {'CL': 'CL:0000127'},
 'CS202212150_5104': {'CL': 'CL:0000127'},
 'CS202212150_5105': {'CL': 'CL:0000127'},
 'CS202212150_5106': {'CL': 'CL:0000127'},
 'CS202212150_5107': {'CL': 'CL:0000127'},
 'CS202212150_5108': {'CL': 'CL:0000127'},
 'CS202212150_5109': {'CL': 'CL:0000127'},
 'CS202212150_5110': {'CL': 'CL:0000127'},
 'CS202212150_5111': {'CL': 'CL:0000127'},
 'CS202212150_5112': {'CL': 'CL:0000127'},
 'CS202212150_5113': {'CL': 'CL:0000127'},
 'CS202212150_5114': {'CL': 'CL:0000127'},
 'CS202212150_5115': {'CL': 'CL:0000127'},
 'CS202212150_5116': {'CL': 'CL:0000127'},
 'CS202212150_5117': {'CL': 'CL:0000127'},
 'CS202212150_5118': {'CL': 'CL:0000127'},
 'CS2022121

In [ ]:
CL_mapping.update({n['Node'] : { 'CL': n['Cell_type']} for n in taxd[0]['Root_nodes']})

In [104]:
CL_mapping

{'CS202212150_4972': {'CL': 'CL:0000120', 'label': 'granule cell'},
 'CS202212150_4973': {'CL': 'CL:0000120', 'label': 'granule cell'},
 'CS202212150_4974': {'CL': 'CL:0000120', 'label': 'granule cell'},
 'CS202212150_4975': {'CL': 'CL:0000120', 'label': 'granule cell'},
 'CS202212150_4976': {'CL': 'CL:0000120', 'label': 'granule cell'},
 'CS202212150_5099': {'CL': 'CL:0000121', 'label': 'Purkinje cell'},
 'CS202212150_5102': {'CL': 'CL:0000644', 'label': 'Bergmann glial cell'},
 'CS202212150_5103': {'CL': 'CL:0000127', 'label': 'astrocyte'},
 'CS202212150_5104': {'CL': 'CL:0000127', 'label': 'astrocyte'},
 'CS202212150_5105': {'CL': 'CL:0000127', 'label': 'astrocyte'},
 'CS202212150_5106': {'CL': 'CL:0000127', 'label': 'astrocyte'},
 'CS202212150_5107': {'CL': 'CL:0000127', 'label': 'astrocyte'},
 'CS202212150_5108': {'CL': 'CL:0000127', 'label': 'astrocyte'},
 'CS202212150_5109': {'CL': 'CL:0000127', 'label': 'astrocyte'},
 'CS202212150_5110': {'CL': 'CL:0000127', 'label': 'astrocyte

In [18]:
#[{'cell_annotation_key': 'neurotransmitter', 'cell_label': 'Glut'},
# {'cell_annotation_key': 'neurotransmitter', 'cell_label': 'GABA'},
# {'cell_annotation_key': 'neurotransmitter', 'cell_label': 'Dopa'},
# {'cell_annotation_key': 'neurotransmitter', 'cell_label': 'Glut-GABA'},
# {'cell_annotation_key': 'neurotransmitter', 'cell_label': 'Chol'},
# {'cell_annotation_key': 'neurotransmitter', 'cell_label': 'Hist'},
# {'cell_annotation_key': 'neurotransmitter', 'cell_label': 'GABA-Glyc'},
# {'cell_annotation_key': 'neurotransmitter', 'cell_label': 'Sero'},
# {'cell_annotation_key': 'neurotransmitter', 'cell_label': 'Nora'},

NT =  { 'Glut': {'ID': 'CL:0000679', 'label': 'glutamatergic neuron'},
        'GABA': {'ID': 'CL:0000617', 'label': 'GABAergic neuron'},
        'Sero': {'ID': 'CL:0000850', 'label': 'serotonergic neuron'},
        'Dopa': {'ID': 'CL:0000700', 'label': 'dopaminergic neuron'},
        'Chol': {'ID': 'CL:0000108', 'label': 'cholinergic neuron'},
        'Hist': {'ID': 'CL:0011110', 'label': 'histaminergic neuron'},
        'Nora': {'ID': 'CL:0008025', 'label': 'noradrendergic neuron        
      }
       ##  TOTO ADD missing NTs

to_remove = []
for k,v in CL_mapping.items():
    if cl_lookup[v['CL']] in [n['label'] for n in NT.values()]:
        to_remove.append(k)
    else: 
        CL_mapping[k]['label'] = cl_lookup[v['CL']]

[CL_mapping.pop(key) for key in to_remove]

list(CL_mapping.items())[0:5]





[('CS202212150_4972', {'CL': 'CL:0000120', 'label': 'granule cell'}),
 ('CS202212150_4973', {'CL': 'CL:0000120', 'label': 'granule cell'}),
 ('CS202212150_4974', {'CL': 'CL:0000120', 'label': 'granule cell'}),
 ('CS202212150_4975', {'CL': 'CL:0000120', 'label': 'granule cell'}),
 ('CS202212150_4976', {'CL': 'CL:0000120', 'label': 'granule cell'})]

In [72]:
cluster_annotation_term_set

,label,name,description,order
0,CCN20230722_NEUR,neurotransmitter,Clusters are assigned based on the average exp...,0
1,CCN20230722_CLAS,class,The top level of cell type definition in the m...,1
2,CCN20230722_SUBC,subclass,The coarse level of cell type definition in th...,2
3,CCN20230722_SUPT,supertype,The second finest level of cell type definitio...,3
4,CCN20230722_CLUS,cluster,The finest level of cell type definition in th...,4


In [19]:
annotation_keys = []
orders = list(cluster_annotation_term_set['order'])
orders.sort()
max_order = orders[-1]
for k,v in cluster_annotation_term_set_dict.items():
    key = {}
    key['annotation_key'] = v['name']
    key['description'] = v['description']
    if not v['name'] == 'neurotransmitter': key['rank'] = - (v['order'] - max_order)
    annotation_keys.append(key)


[{'annotation_key': 'neurotransmitter',
  'description': 'Clusters are assigned based on the average expression of both neurotransmitter transporter genes and key neurotransmitter synthesizing enzyme genes.'},
 {'annotation_key': 'class',
  'description': 'The top level of cell type definition in the mouse whole brain taxonomy. It is primarily determined by broad brain region and neurotransmitter type. All cells within a subclass belong to the same class. Class provides a broader categorization of cell types.',
  'rank': 3}]

In [71]:
pd.DataFrame.from_records(annotation_keys)

,annotation_key,description,rank
0,neurotransmitter,Clusters are assigned based on the average exp...,NaN
1,class,The top level of cell type definition in the m...,3.0
2,subclass,The coarse level of cell type definition in th...,2.0
3,supertype,The second finest level of cell type definitio...,1.0
4,cluster,The finest level of cell type definition in th...,0.0


In [20]:
## Testing matches when old term name is stripped

old_name_new_name_interesction = set(old_taxonomy_dict.keys()).intersection(set([re.sub("^\d+ (.+)", r"\1", v['name']) 
                                                                                    for v in cluster_annotation_term_dict.values()]))
print (len(old_name_new_name_interesction))
print (list(old_name_new_name_interesction)[0:3])

# These names can be used to look up accessions in the old taxonomy

716
['L2/3 IT ENT Glut_6', 'MEA-BST Sox6 Gaba_6', 'VMH Fezf1 Glut_1']


In [21]:
# How many old names have an accession that maps to a CL ID in the root_node_CL_mappings

for name in old_name_new_name_interesction:
    accession = old_taxonomy_dict[name]['cell_set_accession']
    if accession in root_node_CL_mappings.keys():
        id = root_node_CL_mappings[accession]
        print(' ; '.join([id , cl_lookup[id]]))


CL:0000679 ; glutamatergic neuron
CL:0000617 ; GABAergic neuron
CL:0000617 ; GABAergic neuron
CL:4023072 ; brain vascular cell
CL:0000679 ; glutamatergic neuron
CL:0000679 ; glutamatergic neuron
CL:0000679 ; glutamatergic neuron
CL:0000617 ; GABAergic neuron
CL:0000617 ; GABAergic neuron
CL:0000679 ; glutamatergic neuron
CL:0000679 ; glutamatergic neuron
CL:0000617 ; GABAergic neuron
CL:0000679 ; glutamatergic neuron
CL:0000738 ; leukocyte
CL:0000617 ; GABAergic neuron
CL:0000617 ; GABAergic neuron
CL:0000679 ; glutamatergic neuron
CL:0000679 ; glutamatergic neuron
CL:0000700 ; dopaminergic neuron
CL:0000679 ; glutamatergic neuron
CL:0000617 ; GABAergic neuron


In [22]:
# Testing parent cell set lookup

parents = set(cluster_annotation_term['parent_term_label'].fillna(''))
labels = set(cluster_annotation_term['label'].fillna(''))
parents.difference(labels)

{''}

Conclusion - all parent labels resolve internally

In [23]:
from collections import Counter
names = [v['name'] for v in cluster_annotation_term_dict.values()]
duplicate_names = [k for k,v in Counter(names).items() if v>1]

## TODO

* DONE Make more complete set of links to CL, including label - but stripping NT, which should go on NT cell sets.

In [99]:
NTs_only = cluster_to_cluster_annotation_membership.loc[
    cluster_to_cluster_annotation_membership['cluster_annotation_term_set_label'] == 'CCN20230722_NEUR'] 
NTs_only[0:5]

,cluster_annotation_term_label,cluster_annotation_term_set_label,cluster_alias,cluster_annotation_term_name,cluster_annotation_term_set_name,number_of_cells,color_hex_triplet
21288,CS20230722_NEUR_Glut,CCN20230722_NEUR,128,Glut,neurotransmitter,4262,#2B93DF
21289,CS20230722_NEUR_Glut,CCN20230722_NEUR,129,Glut,neurotransmitter,3222,#2B93DF
21290,CS20230722_NEUR_Glut,CCN20230722_NEUR,130,Glut,neurotransmitter,12216,#2B93DF
21291,CS20230722_NEUR_Glut,CCN20230722_NEUR,143,Glut,neurotransmitter,9334,#2B93DF
21292,CS20230722_NEUR_Glut,CCN20230722_NEUR,131,Glut,neurotransmitter,1056,#2B93DF


In [45]:
clusters_only = cluster_to_cluster_annotation_membership.loc[
    cluster_to_cluster_annotation_membership['cluster_annotation_term_set_label'] == 'CCN20230722_CLUS']

In [102]:
clusters_only

,cluster_annotation_term_label,cluster_annotation_term_set_label,cluster_alias,cluster_annotation_term_name,cluster_annotation_term_set_name,number_of_cells,color_hex_triplet
0,CS20230722_CLUS_0001,CCN20230722_CLUS,128,0001 CLA-EPd-CTX Car3 Glut_1,cluster,4262,#00664E
1,CS20230722_CLUS_0002,CCN20230722_CLUS,129,0002 CLA-EPd-CTX Car3 Glut_1,cluster,3222,#5C79CC
2,CS20230722_CLUS_0003,CCN20230722_CLUS,130,0003 CLA-EPd-CTX Car3 Glut_1,cluster,12216,#86FF4D
3,CS20230722_CLUS_0004,CCN20230722_CLUS,143,0004 CLA-EPd-CTX Car3 Glut_1,cluster,9334,#CC563D
4,CS20230722_CLUS_0005,CCN20230722_CLUS,131,0005 CLA-EPd-CTX Car3 Glut_2,cluster,1056,#E7FF26
...,...,...,...,...,...,...,...
5317,CS20230722_CLUS_5318,CCN20230722_CLUS,5279,5318 DC NN_1,cluster,182,#3D5766
5318,CS20230722_CLUS_5319,CCN20230722_CLUS,5275,5319 B cells NN_1,cluster,45,#A673FF
5319,CS20230722_CLUS_5320,CCN20230722_CLUS,5272,5320 ILC NN_2,cluster,28,#3D6659
5320,CS20230722_CLUS_5321,CCN20230722_CLUS,5274,5321 NK cells NN_3,cluster,132,#5CCCA1


In [61]:
import re
annotations = []
import warnings

# In the ols taxonomy only clusters having leading numbers.
def newname_2_old(name:str, old_names):
    old_style_name = re.sub("^\d+ (.+)", r"\1", name)
    if old_style_name in old_names:
        return old_style_name
    else:
        return name

for k,v in cluster_annotation_term_dict.items():
    annotation = {}
    name = newname_2_old(v['name'], old_taxonomy_dict.keys())

    if v['cluster_annotation_term_set_name'] == 'neurotransmitter':
        # Ideally mixed NTs would be split.  Do that later
        annotation['cell_annotation_key'] = v['cluster_annotation_term_set_name']
        annotation['cell_label'] = name
        if name in NT.keys():
            annotation['cell_type'] = NT[name]['label']
            annotation['cell_type_ontology_term_id'] = NT[name]['ID']
            annotation['cell_set_accession'] = v['cluster_annotation_label']
        next
    if name in duplicate_names: 
        next
        
    if name in old_taxonomy_dict.keys():
        annotation['cell_annotation_key'] =  v['cluster_annotation_term_set_name']
        annotation['cell_label'] = name
        cell_set_accession = old_taxonomy_dict[name]['cell_set_accession']
        annotation['cell_set_accession'] = cell_set_accession
        ## Now we need to look up parent cell set accessions. For that we need the stripped name
        ### Lookup name of parent
        if v['parent_term_label'] and v['parent_term_label'] in cluster_annotation_term_dict.keys():
            parent_name = cluster_annotation_term_dict[v['parent_term_label']]['name']
            parent_cell_set_name = newname_2_old(parent_name, old_taxonomy_dict.keys())
            if parent_cell_set_name in old_taxonomy_dict.keys():
                annotation['parent_cell_set_label'] = parent_cell_set_name
                annotation['parent_cell_set_accession'] = [old_taxonomy_dict[parent_cell_set_name]['cell_set_accession']]
            else:
                warnings.warn("No mapping for parent: " + v['parent_term_label'] + ' ' + parent_name)      
        if cell_set_accession in CL_mapping.keys(): 
            annotation['cell_type'] = CL_mapping[cell_set_accession]['label']
            annotation['cell_type_ontology_term_id'] = CL_mapping[cell_set_accession]['CL']
        if v['label'] in cluster_label_2_NT_label.keys():
            annotation['parent_cell_set_accession'].append(cluster_label_2_NT_label[v['label'])
            

    if annotation:
        annotations.append(annotation)


/var/folders/_p/_3x9cz2s5wsbpvwyctdcd_2r0000gq/T/ipykernel_45865/2319846014.py:42: UserWarning: No mapping for parent: CS20230722_CLAS_03 03 OB-CR Glut
  warnings.warn("No mapping for parent: " + v['parent_term_label'] + ' ' + parent_name)
/var/folders/_p/_3x9cz2s5wsbpvwyctdcd_2r0000gq/T/ipykernel_45865/2319846014.py:42: UserWarning: No mapping for parent: CS20230722_CLAS_04 04 DG-IMN Glut
  warnings.warn("No mapping for parent: " + v['parent_term_label'] + ' ' + parent_name)
/var/folders/_p/_3x9cz2s5wsbpvwyctdcd_2r0000gq/T/ipykernel_45865/2319846014.py:42: UserWarning: No mapping for parent: CS20230722_CLAS_06 06 CTX-CGE GABA
  warnings.warn("No mapping for parent: " + v['parent_term_label'] + ' ' + parent_name)
/var/folders/_p/_3x9cz2s5wsbpvwyctdcd_2r0000gq/T/ipykernel_45865/2319846014.py:42: UserWarning: No mapping for parent: CS20230722_CLAS_07 07 CTX-MGE GABA
  warnings.warn("No mapping for parent: " + v['parent_term_label'] + ' ' + parent_name)
/var/folders/_p/_3x9cz2s5wsbpvwyctd

[{'cell_annotation_key': 'subclass',
  'cell_label': 'CA2-FC-IG Glut',
  'cell_set_accession': 'CS202212150_6152',
  'parent_cell_set_label': 'IT-ET Glut',
  'parent_cell_set_accession': 'CS202212150_6347'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L6b EPd Glut',
  'cell_set_accession': 'CS202212150_6145',
  'parent_cell_set_label': 'NP-CT-L6b Glut',
  'parent_cell_set_accession': 'CS202212150_6348'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L6b/CT ENT Glut',
  'cell_set_accession': 'CS202212150_6146',
  'parent_cell_set_label': 'NP-CT-L6b Glut',
  'parent_cell_set_accession': 'CS202212150_6348'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L6b CTX Glut',
  'cell_set_accession': 'CS202212150_6147',
  'parent_cell_set_label': 'NP-CT-L6b Glut',
  'parent_cell_set_accession': 'CS202212150_6348'},
 {'cell_annotation_key': 'subclass',
  'cell_label': 'L6 CT CTX Glut',
  'cell_set_accession': 'CS202212150_6148',
  'parent_cell_set_label': 'NP-CT-L6b Glut',

In [93]:
pd.DataFrame.from_records(annotations[0:10])

,cell_annotation_key,cell_label,cell_type,cell_type_ontology_term_id
0,neurotransmitter,Glut,glutamatergic neuron,CL:0000679
1,neurotransmitter,,NaN,NaN
2,neurotransmitter,GABA,GABAergic neuron,CL:0000617
3,neurotransmitter,Dopa,dopaminergic neuron,CL:0000700
4,neurotransmitter,Glut-GABA,NaN,NaN
5,neurotransmitter,Chol,NaN,NaN
6,neurotransmitter,Hist,NaN,NaN
7,neurotransmitter,GABA-Glyc,NaN,NaN
8,neurotransmitter,Sero,serotonergic neuron,CL:0000850
9,neurotransmitter,Nora,NaN,NaN


In [73]:
pd.DataFrame.from_records([a for a in annotations if ('cell_type' in a.keys())
                        and ('parent_cell_set_accession' in a.keys())])[0:10]

,cell_annotation_key,cell_label,cell_set_accession,parent_cell_set_label,parent_cell_set_accession,cell_type,cell_type_ontology_term_id
0,subclass,VLMC NN,CS202212150_6345,Vascular,CS202212150_6372,vascular leptomeningeal cell,CL:4023051
1,subclass,Peri NN,CS202212150_6118,Vascular,CS202212150_6372,brain pericyte,CL:2000043
2,subclass,SMC NN,CS202212150_6119,Vascular,CS202212150_6372,smooth muscle cell,CL:0000192
3,subclass,Endo NN,CS202212150_6120,Vascular,CS202212150_6372,endothelial cell,CL:0000115
4,subclass,Lymphoid NN,CS202212150_6346,Immune,CS202212150_6373,lymphocyte,CL:0000542
5,supertype,CB Granule Glut_1,CS202212150_6075,CB Granule Glut,CS202212150_6330,granule cell,CL:0000120
6,supertype,Astro-NT NN_1,CS202212150_6098,Astro-NT NN,CS202212150_6337,astrocyte,CL:0000127
7,supertype,Astro-TE NN_3,CS202212150_6100,Astro-TE NN,CS202212150_6338,astrocyte,CL:0000127
8,supertype,Astro-OLF NN_1,CS202212150_6101,Astro-OLF NN,CS202212150_6339,astrocyte,CL:0000127
9,supertype,Tanycyte NN_1,CS202212150_6105,Tanycyte NN,CS202212150_6341,tanycyte,CL:0002085


In [74]:
pd.DataFrame.from_records([a for a in annotations if 
                         ('parent_cell_set_accession' in a.keys())])[0:10]

,cell_annotation_key,cell_label,cell_set_accession,parent_cell_set_label,parent_cell_set_accession,cell_type,cell_type_ontology_term_id
0,subclass,IT EP-CLA Glut,CS202212150_6124,IT-ET Glut,CS202212150_6347,NaN,NaN
1,subclass,L5/6 IT TPE-ENT Glut,CS202212150_6125,IT-ET Glut,CS202212150_6347,NaN,NaN
2,subclass,L6 IT CTX Glut,CS202212150_6126,IT-ET Glut,CS202212150_6347,NaN,NaN
3,subclass,L5 IT CTX Glut,CS202212150_6127,IT-ET Glut,CS202212150_6347,NaN,NaN
4,subclass,L4/5 IT CTX Glut,CS202212150_6128,IT-ET Glut,CS202212150_6347,NaN,NaN
5,subclass,L2/3 IT CTX Glut,CS202212150_6129,IT-ET Glut,CS202212150_6347,NaN,NaN
6,subclass,L2/3 IT ENT Glut,CS202212150_6130,IT-ET Glut,CS202212150_6347,NaN,NaN
7,subclass,IT AON-TT-DP Glut,CS202212150_6132,IT-ET Glut,CS202212150_6347,NaN,NaN
8,subclass,L2 IT ENT-po Glut,CS202212150_6133,IT-ET Glut,CS202212150_6347,NaN,NaN
9,subclass,MEA Slc17a7 Glut,CS202212150_6134,IT-ET Glut,CS202212150_6347,NaN,NaN


In [29]:
# Where are links to NT added? Not in cluster_anntation_term table
cluster_annotation_term['parent_term_set_label'].drop_duplicates()

0                    NaN
44      CCN20230722_CLAS
382     CCN20230722_SUBC
1583    CCN20230722_SUPT
Name: parent_term_set_label, dtype: object

In [30]:
# Seems this happens here & that also Cluster assoc with all levels.  Surely this is redundant!
cluster_to_cluster_annotation_membership['cluster_annotation_term_set_label'].drop_duplicates()

0        CCN20230722_CLUS
5322     CCN20230722_SUPT
10644    CCN20230722_SUBC
15966    CCN20230722_CLAS
21288    CCN20230722_NEUR
Name: cluster_annotation_term_set_label, dtype: object

In [98]:
cluster_to_cluster_annotation_membership[0:5]

,cluster_annotation_term_label,cluster_annotation_term_set_label,cluster_alias,cluster_annotation_term_name,cluster_annotation_term_set_name,number_of_cells,color_hex_triplet
0,CS20230722_CLUS_0001,CCN20230722_CLUS,128,0001 CLA-EPd-CTX Car3 Glut_1,cluster,4262,#00664E
1,CS20230722_CLUS_0002,CCN20230722_CLUS,129,0002 CLA-EPd-CTX Car3 Glut_1,cluster,3222,#5C79CC
2,CS20230722_CLUS_0003,CCN20230722_CLUS,130,0003 CLA-EPd-CTX Car3 Glut_1,cluster,12216,#86FF4D
3,CS20230722_CLUS_0004,CCN20230722_CLUS,143,0004 CLA-EPd-CTX Car3 Glut_1,cluster,9334,#CC563D
4,CS20230722_CLUS_0005,CCN20230722_CLUS,131,0005 CLA-EPd-CTX Car3 Glut_2,cluster,1056,#E7FF26


In [97]:
cluster_to_cluster_annotation_membership.loc[cluster_to_cluster_annotation_membership['cluster_alias'] == 128]

,cluster_annotation_term_label,cluster_annotation_term_set_label,cluster_alias,cluster_annotation_term_name,cluster_annotation_term_set_name,number_of_cells,color_hex_triplet
0,CS20230722_CLUS_0001,CCN20230722_CLUS,128,0001 CLA-EPd-CTX Car3 Glut_1,cluster,4262,#00664E
5322,CS20230722_SUPT_0001,CCN20230722_SUPT,128,0001 CLA-EPd-CTX Car3 Glut_1,supertype,4262,#99822E
10644,CS20230722_SUBC_001,CCN20230722_SUBC,128,001 CLA-EPd-CTX Car3 Glut,subclass,4262,#64c2fc
15966,CS20230722_CLAS_01,CCN20230722_CLAS,128,01 IT-ET Glut,class,4262,#FA0087
21288,CS20230722_NEUR_Glut,CCN20230722_NEUR,128,Glut,neurotransmitter,4262,#2B93DF


## TODO 

- [ ] test redundancy. If cluster maps to term_set of rank n, it should also map to term_set of rank n+1, n+2 etc.
- [ ] Use cluster_annotation_membership to link NTs to clusters (break down dual clusters?) - this will require making accessionsfor NTs.  Maybe just use existing for now.
- [ ] Work out how to integrate neighborhoods - see https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#metadata/WMB-neighborhoods/20230830/
- [ ] Update to latest taxonomy versions (?) s3://aibs-taxonomies/AIT17.0/filename - Need to check version & consider what is gained by doing this.  We also have latest - Could be loaded to taxonomy editor tools.
- [ ] Write up critical doc on ABC schema.  STARTED (see Obsidian)


BINGO

It looks like one can make a cluster ID lookup from this table using only CCN20230722_CLUS rows.  
In this case cluster 128 = 0001 CLA-EPd-CTX Car3 Glut_1	 CS20230722_CLUS_0001 so we can assign that as Glut.

So we can make a lookups alias -> cluster_annotation_term_label and cluster_annotation_term_label -> NT

In some ways - the cluster alias system is there version of an accession.   Where our system allows multiple parent mappings.

In [96]:
cluster_to_cluster_annotation_membership.loc[cluster_to_cluster_annotation_membership['cluster_annotation_term_set_label'] == "CCN20230722_NEUR"]


,cluster_annotation_term_label,cluster_annotation_term_set_label,cluster_alias,cluster_annotation_term_name,cluster_annotation_term_set_name,number_of_cells,color_hex_triplet
21288,CS20230722_NEUR_Glut,CCN20230722_NEUR,128,Glut,neurotransmitter,4262,#2B93DF
21289,CS20230722_NEUR_Glut,CCN20230722_NEUR,129,Glut,neurotransmitter,3222,#2B93DF
21290,CS20230722_NEUR_Glut,CCN20230722_NEUR,130,Glut,neurotransmitter,12216,#2B93DF
21291,CS20230722_NEUR_Glut,CCN20230722_NEUR,143,Glut,neurotransmitter,9334,#2B93DF
21292,CS20230722_NEUR_Glut,CCN20230722_NEUR,131,Glut,neurotransmitter,1056,#2B93DF
...,...,...,...,...,...,...,...
26605,CS20230722_NEUR_NA,CCN20230722_NEUR,5279,NaN,neurotransmitter,182,#666666
26606,CS20230722_NEUR_NA,CCN20230722_NEUR,5275,NaN,neurotransmitter,45,#666666
26607,CS20230722_NEUR_NA,CCN20230722_NEUR,5272,NaN,neurotransmitter,28,#666666
26608,CS20230722_NEUR_NA,CCN20230722_NEUR,5274,NaN,neurotransmitter,132,#666666


Above has multiple rows recording glutamatergic clusters.  The only mapping to a cluster ID appears to be via something called cluster_alias.  Searching the entire repo for this finds mention in a couple of notebooks. Working through the logic of pivot tables etc is hard.



In [59]:
cluster_2_NT = df2dict(NTs_only, key_column = 'cluster_alias')

In [49]:
# Check that cluster_alias => unique key
aliases = list(clusters_only['cluster_alias'])
[k for k,v in Counter(aliases).items() if v>1]


[]

In [85]:
cluster_lookup = df2dict(clusters_only, key_column = 'cluster_alias')

In [86]:
cluster_lookup

{128: {'cluster_annotation_term_label': 'CS20230722_CLUS_0001',
  'cluster_annotation_term_set_label': 'CCN20230722_CLUS',
  'cluster_annotation_term_name': '0001 CLA-EPd-CTX Car3 Glut_1',
  'cluster_annotation_term_set_name': 'cluster',
  'number_of_cells': 4262,
  'color_hex_triplet': '#00664E'},
 129: {'cluster_annotation_term_label': 'CS20230722_CLUS_0002',
  'cluster_annotation_term_set_label': 'CCN20230722_CLUS',
  'cluster_annotation_term_name': '0002 CLA-EPd-CTX Car3 Glut_1',
  'cluster_annotation_term_set_name': 'cluster',
  'number_of_cells': 3222,
  'color_hex_triplet': '#5C79CC'},
 130: {'cluster_annotation_term_label': 'CS20230722_CLUS_0003',
  'cluster_annotation_term_set_label': 'CCN20230722_CLUS',
  'cluster_annotation_term_name': '0003 CLA-EPd-CTX Car3 Glut_1',
  'cluster_annotation_term_set_name': 'cluster',
  'number_of_cells': 12216,
  'color_hex_triplet': '#86FF4D'},
 143: {'cluster_annotation_term_label': 'CS20230722_CLUS_0004',
  'cluster_annotation_term_set_labe

In [55]:
NTs_only

,cluster_annotation_term_label,cluster_annotation_term_set_label,cluster_alias,cluster_annotation_term_name,cluster_annotation_term_set_name,number_of_cells,color_hex_triplet
21288,CS20230722_NEUR_Glut,CCN20230722_NEUR,128,Glut,neurotransmitter,4262,#2B93DF
21289,CS20230722_NEUR_Glut,CCN20230722_NEUR,129,Glut,neurotransmitter,3222,#2B93DF
21290,CS20230722_NEUR_Glut,CCN20230722_NEUR,130,Glut,neurotransmitter,12216,#2B93DF
21291,CS20230722_NEUR_Glut,CCN20230722_NEUR,143,Glut,neurotransmitter,9334,#2B93DF
21292,CS20230722_NEUR_Glut,CCN20230722_NEUR,131,Glut,neurotransmitter,1056,#2B93DF
...,...,...,...,...,...,...,...
26605,CS20230722_NEUR_NA,CCN20230722_NEUR,5279,NaN,neurotransmitter,182,#666666
26606,CS20230722_NEUR_NA,CCN20230722_NEUR,5275,NaN,neurotransmitter,45,#666666
26607,CS20230722_NEUR_NA,CCN20230722_NEUR,5272,NaN,neurotransmitter,28,#666666
26608,CS20230722_NEUR_NA,CCN20230722_NEUR,5274,NaN,neurotransmitter,132,#666666


In [57]:
# Check each cluster has only one NT assignement
aliases = list(NTs_only['cluster_alias'])
[k for k,v in Counter(aliases).items() if v>1]

[]

In [60]:
cluster_2_NT[128]

{'cluster_annotation_term_label': 'CS20230722_NEUR_Glut',
 'cluster_annotation_term_set_label': 'CCN20230722_NEUR',
 'cluster_annotation_term_name': 'Glut',
 'cluster_annotation_term_set_name': 'neurotransmitter',
 'number_of_cells': 4262,
 'color_hex_triplet': '#2B93DF'}

In [87]:
cluster_label_2_NT = { cluster_lookup[k]['cluster_annotation_term_label'] : v['cluster_annotation_term_name']
                      for k,v in cluster_2_NT.items()  }

In [88]:
cluster_label_2_NT 

{'CS20230722_CLUS_0001': 'Glut',
 'CS20230722_CLUS_0002': 'Glut',
 'CS20230722_CLUS_0003': 'Glut',
 'CS20230722_CLUS_0004': 'Glut',
 'CS20230722_CLUS_0005': 'Glut',
 'CS20230722_CLUS_0006': 'Glut',
 'CS20230722_CLUS_0007': 'Glut',
 'CS20230722_CLUS_0008': 'Glut',
 'CS20230722_CLUS_0009': 'Glut',
 'CS20230722_CLUS_0010': 'Glut',
 'CS20230722_CLUS_0011': 'Glut',
 'CS20230722_CLUS_0012': 'Glut',
 'CS20230722_CLUS_0013': 'Glut',
 'CS20230722_CLUS_0014': 'Glut',
 'CS20230722_CLUS_0015': 'Glut',
 'CS20230722_CLUS_0016': 'Glut',
 'CS20230722_CLUS_0017': 'Glut',
 'CS20230722_CLUS_0018': 'Glut',
 'CS20230722_CLUS_0019': 'Glut',
 'CS20230722_CLUS_0020': 'Glut',
 'CS20230722_CLUS_0021': 'Glut',
 'CS20230722_CLUS_0022': 'Glut',
 'CS20230722_CLUS_0023': 'Glut',
 'CS20230722_CLUS_0024': 'Glut',
 'CS20230722_CLUS_0025': 'Glut',
 'CS20230722_CLUS_0026': 'Glut',
 'CS20230722_CLUS_0027': 'Glut',
 'CS20230722_CLUS_0028': 'Glut',
 'CS20230722_CLUS_0029': 'Glut',
 'CS20230722_CLUS_0030': 'Glut',
 'CS202307

In [84]:
clusters_only[['cluster_annotation_term_label', 'cluster_alias', 'cluster_annotation_term_name']]

,cluster_annotation_term_label,cluster_alias,cluster_annotation_term_name
0,CS20230722_CLUS_0001,128,0001 CLA-EPd-CTX Car3 Glut_1
1,CS20230722_CLUS_0002,129,0002 CLA-EPd-CTX Car3 Glut_1
2,CS20230722_CLUS_0003,130,0003 CLA-EPd-CTX Car3 Glut_1
3,CS20230722_CLUS_0004,143,0004 CLA-EPd-CTX Car3 Glut_1
4,CS20230722_CLUS_0005,131,0005 CLA-EPd-CTX Car3 Glut_2
...,...,...,...
5317,CS20230722_CLUS_5318,5279,5318 DC NN_1
5318,CS20230722_CLUS_5319,5275,5319 B cells NN_1
5319,CS20230722_CLUS_5320,5272,5320 ILC NN_2
5320,CS20230722_CLUS_5321,5274,5321 NK cells NN_3
